In [1]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from case_edits.recipes.weather import input
from case_edits.ezcase import EzCase

from epw import epw

import seaborn as sns

import pandas as pd
import numpy as np

from weather.joint_analysis import JointAnalysis, N_DATETIME_COLS
from weather.irrelevant_quantities import irrelevant_qois
import plotly.express as px

import plotly.graph_objects as go
import datetime

In [102]:
ja = JointAnalysis()

ja.update_qois(["Dry Bulb Temperature", "Wind Speed" ])#"Wind Direction"
ja.create_histogram_df()
ja.freqs_df.head()

df = ja.summer_df
df["Date"] = pd.to_datetime(
            ja.summer_df[["Year", "Month", "Day"]]
        )
df["Distribution Groups"] = np.full(len(df), -99)

In [85]:
# filter to find when db between desired.. 
mask = (df["Dry Bulb Temperature"] > 15.47	) & (df["Dry Bulb Temperature"] < 18.93) & (df["Wind Speed"] > 1.97	) & (df["Wind Speed"] < 3.93)
# df_joint = df.loc[mask]
# df_joint = df_joint.reset_index(drop=True)
mask.value_counts()

False    1225
True      263
dtype: int64

In [103]:
mask = True

# ignore "Occurences count"
freqs_df = ja.freqs_df.iloc[:, :-1]

for ix in list(range(ja.nfreqs)):
    for key, value in freqs_df.loc[(ix, "min")].iteritems(): 
        mask &= df[key] > value
    for key, value in freqs_df.loc[(ix, "max")].iteritems(): 
        mask &= df[key] < value
    df.loc[mask, "Distribution Groups"] = ix
    mask = True

In [101]:
df["Distribution Groups"].unique()

array([ 1,  2, -1,  0,  3,  7,  9,  5,  4,  6,  8])

In [104]:
hours = list(df["Hour"].unique())
dates = list(df["Date"].unique())
arr = np.zeros((len(hours), len(dates)))

for ix, hour in enumerate(hours):
    mask = df["Hour"] == hour
    df1 = df.loc[mask]
    res = df1.reset_index(drop=True)["Distribution Groups"]
    arr[ix, :] = res


In [105]:
fig = go.Figure(data=go.Heatmap(
        z=arr,
        x=dates,
        y=hours,
        colorscale='Viridis'))
fig.show()

In [ ]:
# find all hours and days in df_joint, and make them red here => give them a strange z value